In [ ]:
from larch.xafs import feffit, feffit_dataset, feffit_transform, feffit_report
from larch.xafs import pre_edge, autobk, xftf, xftr, ff2chi, feffpath
from larch.fitting import  param_group, param
from larch.io import h5group, read_ascii, read_athena, read_xdi, read_specfile, extract_athenagroup

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
""" 2.1 Setup feffit - feffit parameters + FT and fitting space """

# Create feffit Parameter Group to hold fit parameters (Core parameters)
_feffit_params = param_group(s02=param(0.87348, min=0, vary=False),
                             e0=param(0.1, min=-10, max=10, vary=True),
#                              sigma2_cu=param(0.008, min=0.001, max=0.009, vary=True),
                             sigma2_o=param(0.008, min=0.001, max=0.009, vary=True))

# Create feffit Parameter Group to hold fit parameters (Repeating parameters)
data_num = len(data_list)

fit_params = ["n_cuo_", "delr_o_"]
fit_values = [param(1.0, vary=True),
              param(0.001, min=-0.5, max=0.5, vary=True)]

all_fit_params = [param + str(i) for i in range(data_num) for param in fit_params]
all_fit_values = fit_values * data_num

for key, value in zip(all_fit_params, all_fit_values):
    setattr(_feffit_params, key, value)

# Define Fourier transform and fitting space
# Actual FT parameters for EXAFS fitting
_feffit_trans = feffit_transform(kmin=3.000, kmax=10.000, dk=1.0000, kw=2,
                                 window="hanning", fitspace='r', rmin=1.000, rmax=4.000)


In [ ]:
""" 2.2 Setup up paths for all datasets. """

# Create list of base paths
_paths = []

_paths_names = ["O195_"]
_all_paths_names = [pname + str(i) for i in range(data_num) for pname in _paths_names]

path_original = "C:/Users/user/larch/feff/Cu1_K_Tenorite_cif11639/feff0001.dat"
path_new = "C:/Users/user/Downloads/Cu-O_path_at_2_1500.rsp"

_path_o = feffpath(path_new,
               degen=1, e0='e0', sigma2='sigma2_o')

_path_keys = ["label", "s02", "deltar"]
_path_values_o = ["O195_", "s02 * n_cuo_", "delr_o_"]

for i in range(data_num):

    # Setup for Cu-O paths    
    _path_o_temp = copy.deepcopy(_path_o)
    for key, value in zip(_path_keys, _path_values_o):
        setattr(_path_o_temp, key, value + str(i))
        
    _paths_dict = {_paths_names[0] + str(i): _path_o_temp}
    
    _paths.append(_paths_dict)